In [ ]:
class Stack:
    def __init__(self):
        self._stack = []

    def size(self):
        return len(self._stack)

    def top(self):
        return self._stack[-1]

    def push(self, item):
        # Time: O(1)
        self._stack.append(item)

    def pop(self):
        last_element = self._stack[-1]
        self._stack.pop()
        return last_element

if __name__ == '__main__':
    s = Stack()
    print(s.size())
    s.push(5)
    print(s.size())
    print(s.top())


In [8]:
import random
import math

class HashTable:
    class HashElement:
        def __init__(self, key, value):
            self._key = key
            self._value = value
        
        def get_key(self):
            return self._key
        
        def get_value(self):
            return self._value
        
    def __init__(self, size):
        self._capacity = size
        self._load_factor = .75
        self._size = 0
        self._hash_table = [[None]]*self._capacity
        self._iv = random.getrandbits(8)

    def _mix(self, block, internal_state):
        return (internal_state * block) ^ ((internal_state << 2) + (block >> 1))

    def _hash(self, item):
        '''Utilizes Merkle-Damgard construction (collision-resistant cryptographic hash)'''
        internal_state = self._iv
        blocks = []
        # Convert item to binary and break up into 8-bit blocks
        if type(item) == int:
            temp = item
            while temp > 0:
                block = (temp & 0xff)
                temp >>= 8
                blocks.append(format(block, '08b'))
        elif type(item) == str:
            blocks = [format(ord(char), '08b') for char in item]
        else:
            print('err: invalid type -- key must be \'int\' or \'str\'')
            return
        
        # Mix blocks to produce the final output
        for block in blocks:
            mixable_block = int(block, 2)
            internal_state = self._mix(mixable_block, internal_state)

        return internal_state

    def get(self, key):
        '''Returns the value of a given key'''
        # Time: O(1)
        # Space: O(1)
        index = self._hash(key) % self._capacity
        
        if index > self._capacity:
            print('err: hashed index out of bounds')
            return
        
        hash_elements = self._hash_table[index]
        print(hash_elements)
        for element in hash_elements:
            if element == None:
                continue
    
            if element.get_key() == key:
                return element.get_value()
        
        print('err: element could not be found for key: {}'.format(key))
        return None

    def _resize(self):
        '''Increases the size of the hash table'''
        # Time: O(size(hash_table))
        # Space: O(2*capacity(hash_table))
        self._capacity = math.ceil(self._capacity*1.5)
        new_hash_table = [[None]]*self._capacity

        for i in range(len(self._hash_table)):
            if self._hash_table[i] == [None]:
                continue
            
            hash_elements = self._hash_table[i]
            for element in hash_elements:
                new_index = self._hash(element.get_key()) % self._capacity
                new_hash_table[new_index] += [element]

        self._hash_table = new_hash_table

    def put(self, key, value):
        '''Maps an item into an index in the hash table'''
        # Time: amortized O(n)
        index = self._hash(key) % self._capacity
        if self._hash_table[index] == [None]:
            self._hash_table[index] = [HashTable.HashElement(key, value)]
        else:
            self._hash_table[index] += [HashTable.HashElement(key, value)]

        self._size += 1

        # If size has met or exceeded the load capacity, grow the table
        if self._size / self._capacity >= self._load_factor:
            self._resize()

if __name__ == '__main__':
    ht = HashTable(100)
    ht.put(123456, 'dogs')
    ht.put('secret secret..', 'are')
    ht.put('yoyo', 'cute')

    assert 'dogs' == ht.get(123456)
    assert 'are' == ht.get('secret secret..')
    assert 'cute' == ht.get('yoyo')

    ht = HashTable(2)
    ht.put(123, 'boop boop')
    ht.put('abc', 'beep beep')

    assert 'boop boop' == ht.get(123)
    assert 'beep beep' == ht.get('abc')

[None, <__main__.HashTable.HashElement object at 0x0120E150>, <__main__.HashTable.HashElement object at 0x0120EC50>]
[None, <__main__.HashTable.HashElement object at 0x0120E150>, <__main__.HashTable.HashElement object at 0x0120EC50>]


In [23]:
import queue
import math

class MaxHeap:
    def __init__(self):
        self._heap = []

    def add(self, item):
        '''Add an element to the heap, maintaining the heap structure'''
        # Time complexity: O(logn)
        # Space complexity: O(1)

        self._heap.append(item)
        self._heapify_up(len(self._heap) - 1)
    
    def _heapify_up(self, index):
        parent_index = math.floor((index - 1) / 2)
        if parent_index > -1 and self._heap[index] > self._heap[parent_index]:
                self._heap[index], self._heap[parent_index] = self._heap[parent_index], self._heap[index]
                self._heapify_up(parent_index)
                
    def _heapify_down(self, index):
        largest = index
        left = 2*index + 1
        right = 2*index + 2
        
        if left < len(self._heap) and self._heap[largest] < self._heap[left]:
            largest = left
            
        if right < len(self._heap) and self._heap[largest] < self._heap[right]:
            largest = right
        
        if largest == index:
            return
        
        self._heap[index], self._heap[largest] = self._heap[largest], self._heap[index]
        self._heapify_down(largest)

    def pop(self):
        '''Removes the top of the heap (the max element)'''
        # Time: O(n) due to implementation of having max being the first element
        # - would be O(logn) otherwise
        # Space: O(1)

        max_element = self._heap[0]
        self._heap.pop(0)
        self._heapify_down(0)
        return max_element

    def print_heap(self):
        '''Prints the levels of the heap in level-order'''
        # Time: O(b^m) where m is the maximum level
        # Space: O(bm)

        if len(self._heap) < 1:
            return

        q = queue.Queue()
        q.put(0)
        print(self._heap[0])

        while not q.empty():
            level = []
            rootIndex = q.get()
            left_child_index = 2*rootIndex + 1
            right_child_index = 2*rootIndex + 2

            if left_child_index < len(self._heap):
                q.put(left_child_index)
                level.append(self._heap[left_child_index])

            if right_child_index < len(self._heap):
                q.put(right_child_index)
                level.append(self._heap[right_child_index])

            for item in level:
                print(item, end=' ')
            print()

if __name__ == '__main__':
    mh = MaxHeap()
    mh.add(1)
    mh.add(3)
    mh.add(5)
    mh.add(7)
    mh.add(2)
    mh.print_heap()

    assert mh.pop() == 7
    assert mh.pop() == 5
    assert mh.pop() == 3
    assert mh.pop() == 2
    assert mh.pop() == 1


7
5 3 
1 2 



